## Import statements

In [1]:
import plate_utils
import sys
import os
import glob
import numpy as np
import pandas as pd
import multiprocessing as mp
from astropy.io import fits, ascii
from astropy import wcs
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column
import pathlib
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'SciServer'

### Specify data path

In [ ]:
datapath = '../Methods Paper/'
pathlib.Path(datapath + 'data_tables').mkdir(exist_ok=True)

### Display files in path

In [ ]:
files = glob.glob(datapath + '*.fit*')
files = [os.path.basename(path) for path in files]
plate_utils.index_print(files)

### Print name of current file

In [ ]:
whichfile = 1
extensionless_name = files[whichfile][:files[whichfile].find('.')]
print('whichfile =',files[whichfile])

In [ ]:
Ry14 = plate_utils.Plate(datapath + files[whichfile])
Ry14.crop((1690, 2007), 500)
#Ry14.invert()
Ry14.data[400:600, 400:600] = np.median(Ry14.data)
plate_utils.display(Ry14.data)

In [ ]:
import time
start_proc = time.time()
p = mp.Process(target=Ry14.query(threshold=20))
start_fast = time.time()
print("Proc: ", start_fast - start_proc)
p.start()
Ry14.sextract()
p.join()
print("Quextract: ",time.time() - start_proc)
start_match_fast = time.time()
Ry14.match()
print(time.time() - start_match_fast)

In [ ]:
#Ry14.center = (1750, 1650)
Ry14.table['center_dist'] = ((Ry14.table['plate_x'] - Ry14.center[0])**2 + (Ry14.table['plate_y'] - Ry14.center[1])**2)**0.5

In [ ]:
Ry14 = plate_utils.Plate

In [ ]:
Ry14.table = pd.read_csv('../Methods Paper/Ry60_rgk.csv')

In [ ]:
print(Ry14.table)

In [ ]:
Ry14.table['umg'] = Ry14.table['u'] - Ry14.table['g']
Ry14.table['gmp'] = Ry14.table['g'] - Ry14.table['mag_plate']

In [ ]:
Ry14.table['res'] = Ry14.table['pg'] - Ry14.table['mag']
Ry14.table['sqrt_signed'] = (np.abs(Ry14.table['res']) ** 0.5) * np.sign(Ry14.table['res'])
Ry14.table['log_res_abs'] = np.log(np.abs(Ry14.table['res']))
pos_res_table = Ry14.table[Ry14.table['res'] > 0]
neg_res_table = Ry14.table[Ry14.table['res'] < 0]

In [ ]:
plt.hist(Ry14.table['res'], bins='scott')

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.gca().invert_xaxis()
plt.gca().set_ylabel('Dec')
plt.gca().set_xlabel('RA')
plt.title('Residuals of all objects (sqrt scale)')
plt.scatter(Ry14.table['ra'], Ry14.table['dec'],
            c=Ry14.table['sqrt_signed'])
plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.gca().invert_xaxis()
plt.gca().set_ylabel('Dec')
plt.gca().set_xlabel('RA')
plt.title('Residuals of all objects (log scale)')
plt.scatter(pos_res_table['ra'], pos_res_table['dec'],
            c=pos_res_table['log_res_abs'])
plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.gca().invert_xaxis()
plt.gca().set_ylabel('Dec')
plt.gca().set_xlabel('RA')
plt.title('Residuals of all objects (log scale)')
plt.scatter(neg_res_table['ra'], neg_res_table['dec'],
            c=np.array(neg_res_table['log_res_abs']))
plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.gca().invert_xaxis()
plt.gca().set_ylabel('Dec')
plt.gca().set_xlabel('RA')
plt.title('Residuals of all objects (log scale)')
new_table = Ry14.table
plt.scatter(new_table['ra'], new_table['dec'],
            c=np.array(new_table['res']))
plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.ylim(-2, 2)
plt.gca().set_xlabel('Distance from center')
plt.gca().set_ylabel('pg - mag')
plt.title('Residuals of all objects')
new_table = Ry14.table
plt.scatter(new_table['rad'], new_table['res'], c=new_table['pg'])
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(11.5, 11.5))
# plt.ylim(5.3, 6.4)
# plt.xlim(14.9, 18)
plt.gca().invert_yaxis()
plt.gca().set_ylabel('log10(flux)')
plt.gca().set_xlabel('SDSS g magnitudes')
plt.scatter(Ry14.table['g'], Ry14.table['mag_plate'], c=Ry14.table['center_dist'])

In [ ]:
plt.figure(figsize=(11.5, 11.5))
plt.gca().set_ylabel('u - g')
plt.gca().set_xlabel('g - plate')
# plt.xlim(0.75, 2)
#plt.ylim(7, 10)
# plt.gca().set_ylabel('u - g')
# plt.gca().set_xlabel('g - plate')
plt.scatter(Ry14.table['umg'], Ry14.table['gmp'])

In [ ]:
def color_correction(x):
    h1 = 1
    k1 = 4
    h2 = 2.5
    k2 = 3.2
    slope = (k2 - k1) / (h2 - h1)
    return (slope*(h1 - x)) + k1

Ry14.table['color_correction'] = color_correction(np.array(Ry14.table['g']))
Ry14.table['corrected_mags'] = Ry14.table['mag_plate'] - Ry14.table['color_correction']

plt.figure(figsize=(11.5, 11.5))
# plt.ylim(5.3, 6.4)
# plt.xlim(14.9, 18)
plt.gca().invert_yaxis()
plt.gca().set_ylabel('SDSS g magnitudes')
plt.gca().set_xlabel('log10(flux)')
plt.scatter(Ry14.table['corrected_mags'], Ry14.table['g'], c=Ry14.table['center_dist'])

In [ ]:
import scipy.stats
slope_cal, intercept_cal, r_value, p_value, std_err = scipy.stats.linregress(Ry14.table['corrected_mags'], Ry14.table['g'])
print(std_err)
def calibrate_mag(x):
    return (slope_cal * x) + intercept_cal

In [ ]:
# 1: 167.86885, 28.77947
# 2: 167.59673, 28.73918
# 3: 167.68761, 28.61742

three_coords = [(167.86876, 28.77950), (167.68729, 28.61764), (167.59663, 28.73876)]
three_ids = star_info_SDSS['objid'][quick_match(three_coords, star_info_SDSS_coords, 0.1, 0.5)]
three_mags = np.array([Ry14.table['corrected_mags'][np.where(Ry14.table['objid'] == i)][0] for i in three_ids])

print(three_mags)
print(calibrate_mag(three_mags))

### Gaussian fitting

In [ ]:
domain_half = 20
d_range = range(0 - domain_half, domain_half)
slices = []
gaussian_fits = []
for i in range(len(star_cm_locations)):
    if((star_x_cm_location[i] < (len(image_data[0]) - domain_half)) and (star_x_cm_location[i] > domain_half)):
        star_slice = image_data[int(star_y_cm_location[i])][int(star_x_cm_location[i] - domain_half) : int(star_x_cm_location[i] + domain_half)]
        slices.append(star_slice)
        params, params_cov = opt.curve_fit(gaussian_x, d_range, star_slice, [10000, 0, 1, plate_median])
        gaussian_fits.append(params)
gaussian_fits = np.asarray(gaussian_fits)

### CSV writing

In [ ]:
table_len = len(cleaned_gaia_mags)
preprocessing_table = Table()
# preprocessing_table['plate_median'] = np.full(table_len, plate_median)
# preprocessing_table['plate_width'] = np.full(table_len, plate_width)
# preprocessing_table['plate_width_wcs'] = np.full(table_len, plate_width_wcs)
# preprocessing_table['g_amplitude'] = gaussian_fits[:, 0]
# preprocessing_table['g_width'] = gaussian_fits[:, 2]
# preprocessing_table['g_background'] = gaussian_fits[:, 3]
preprocessing_table['gaia_flux'] = star_fluxes
preprocessing_table['gaia_mag'] = star_info_SDSS['g']
# preprocessing_table['SDSS_flux'] = star_fluxes
# preprocessing_table['SDSS_mag'] = star_info_SDSS['g']

preprocessing_table.write(datapath + '/data_tables/' + extensionless_name + '.csv', format='csv', overwrite=True)